## **Flow of the HITL**
```
START
  ↓
step_A        ← automated
  ↓
⏸ PAUSE       ← waiting for human
  ↓
human_feedback (manual update)
  ↓
step_B        ← automated
  ↓
step_C        ← automated
  ↓
END

```

In [1]:
# -------------------------
# Import required libraries
# -------------------------

# BaseModel is used to define a strongly-typed state with validation
from pydantic import BaseModel

# Optional allows fields to be nullable
from typing import Optional

# StateGraph is used to build the LangGraph workflow
# START and END define entry and exit points of the graph
from langgraph.graph import StateGraph, START, END

# MemorySaver stores graph state so execution can pause and resume
from langgraph.checkpoint.memory import MemorySaver


# =========================================================
# 1. STATE DEFINITION
# =========================================================
# This class defines the global state shared across all nodes.
# Every node receives this state and must return it (or updates to it).

class State(BaseModel):
    # Initial input provided when the graph starts
    input: str

    # Human feedback collected during execution
    # Optional so the graph can start without it
    user_feedback: Optional[str] = None


# =========================================================
# 2. NODE DEFINITIONS
# =========================================================
# Each function below is a node in the LangGraph.
# A node:
# - Receives the current state
# - Performs some logic
# - Returns the (updated) state


def step_A(state: State) -> State:
    """
    First automated step in the workflow.
    Executes immediately after START.
    """
    print("Step_A executed")

    # We are not modifying the state here,
    # so we simply return it as-is
    return state


def human_feedback(state: State) -> State:
    """
    Human-in-the-loop node.
    Execution will PAUSE before this node runs.
    """
    print("Human Feedback Node")

    # This code will NOT execute initially because
    # interrupt_before=["human_feedback"] is enabled
    return state


def step_B(state: State) -> State:
    """
    Executes after human feedback is provided.
    """
    print("Step_B executed")
    print("User feedback received:", state.user_feedback)

    return state


def step_C(state: State) -> State:
    """
    Final processing step before ending the graph.
    """
    print("Step_C executed")
    print("Final user feedback:", state.user_feedback)

    return state


# =========================================================
# 3. GRAPH CONSTRUCTION
# =========================================================
# StateGraph connects nodes together and defines execution flow.

builder = StateGraph(State)

# Register nodes with the graph
builder.add_node("step_A", step_A)
builder.add_node("human_feedback", human_feedback)
builder.add_node("step_B", step_B)
builder.add_node("step_C", step_C)

# Define the execution order using edges
builder.add_edge(START, "step_A")                 # Start → Step A
builder.add_edge("step_A", "human_feedback")      # Step A → Human Feedback
builder.add_edge("human_feedback", "step_B")      # Feedback → Step B
builder.add_edge("step_B", "step_C")              # Step B → Step C
builder.add_edge("step_C", END)                   # Step C → End


# =========================================================
# 4. CHECKPOINTING (MEMORY)
# =========================================================
# MemorySaver allows the graph to:
# - Pause execution
# - Resume later from the exact same point

memory = MemorySaver()

# Compile the graph with interruption enabled
graph = builder.compile(
    checkpointer=memory,
    interrupt_before=["human_feedback"]  # Pause before human_feedback node
)


# =========================================================
# 5. INITIAL INPUT
# =========================================================
# This is the input state provided when execution starts

initial_input = {
    "input": "Hello World"
}

# Thread ID identifies a unique execution session
thread = {
    "configurable": {
        "thread_id": "1"
    }
}


# =========================================================
# 6. RUN GRAPH UNTIL INTERRUPTION
# =========================================================
# The graph will execute nodes until it reaches
# the human_feedback node, then pause.

print("\n--- Running graph until interruption ---\n")

for event in graph.stream(initial_input, thread, stream_mode="values"):
    # Each event represents a state update
    print(event)


# =========================================================
# 7. CHECK STATE BEFORE HUMAN INPUT
# =========================================================
# At this point:
# - step_A has executed
# - execution is paused before human_feedback

print("\n--- State before human feedback ---")
print(graph.get_state(thread).values)


# =========================================================
# 8. COLLECT HUMAN INPUT
# =========================================================
# Simulate a real human giving feedback

user_input = input("\nTell me how you want to update the state: ")

# Update the graph state manually
# as_node="human_feedback" tells LangGraph:
# "Pretend this update came from that node"

graph.update_state(
    thread,
    {"user_feedback": user_input},
    as_node="human_feedback"
)


# =========================================================
# 9. VERIFY UPDATED STATE
# =========================================================
print("\n--- State after update ---")
print(graph.get_state(thread).values)

print("\nNext node to execute:")
print(graph.get_state(thread).next)


# =========================================================
# 10. RESUME GRAPH EXECUTION
# =========================================================
# Continue execution from where it was paused

print("\n--- Resuming graph execution ---\n")

for event in graph.stream(None, thread, stream_mode="values"):
    print(event)


# =========================================================
# 11. FINAL STATE
# =========================================================
# Graph has fully completed execution

print("\n--- Final State ---")
print(graph.get_state(thread).values)



--- Running graph until interruption ---

{'input': 'Hello World'}
Step_A executed
{'input': 'Hello World'}

--- State before human feedback ---
{'input': 'Hello World'}



Tell me how you want to update the state:  approved



--- State after update ---
{'input': 'Hello World', 'user_feedback': 'approved'}

Next node to execute:
('step_B',)

--- Resuming graph execution ---

{'input': 'Hello World', 'user_feedback': 'approved'}
Step_B executed
User feedback received: approved
{'input': 'Hello World', 'user_feedback': 'approved'}
Step_C executed
Final user feedback: approved
{'input': 'Hello World', 'user_feedback': 'approved'}

--- Final State ---
{'input': 'Hello World', 'user_feedback': 'approved'}
